In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
%matplotlib inline
plt.style.use('bmh')

In [2]:
#!pip install tensorflow
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train.npy')

labels_names =['Stadium','Building','Traffic Sign','Forest','Flowers',
              'Street','Classroom','Bridge','Statue','Lake']

print(data_train.shape, labels_train.shape)

(270000, 3124) (3124,)


In [4]:
! pip install  opencv-python
import cv2

In [5]:
ll = 5 # class
bins = 64 # bins for the histogram
print('Class ',labels_names[ll-1])

# Parameters for plotting
kk=[0,3,6]
channels=['Red','Green','Blue']
cc=['r','g','b']

# Permutes the index locations for class ll
rnd_sample = npr.permutation(np.where(labels_train==ll)[0])

# Plot the histogram for the first 10 images
# for i in range(10):
#     image = data_train[:,rnd_sample[i]].reshape((300,300,3))
#     plt.figure(figsize=(15,5))
#     plt.subplot(1,2,1); plt.imshow(image); plt.axis('off')
#     plt.subplot(1,2,2)
#     for j in range(3):
#         channel = image[:,:,j]
#         hist = cv2.calcHist([channel], [0], None, [bins], [0, 256])
#         plt.plot(hist,label=channels[j],c=cc[j]); plt.xlim([0, bins])
#         plt.ylabel('# of Pixels')
#     plt.xlabel('Bins');plt.legend();plt.show();

Class  Flowers


In [6]:
# Obtain the Histogram Features for all Samples
features = []
for i in range(data_train.shape[1]):
    image = data_train[:,i].reshape((300,300,3))
    f_hist=[]
    for j in range(3):
        channel = image[:,:,j]
        hist = cv2.calcHist([channel], [0], None, [32], [0, 256])
        if j ==0:
            f_hist=hist
        else:
            f_hist=np.vstack((f_hist,hist))
    if i==0:
        features = f_hist
    else:
        features = np.hstack((features, f_hist))
features = features.T
print(features.shape)

print(data_train.shape)

(3124, 96)
(270000, 3124)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(data_train.transpose(), labels_train, test_size=0.20)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
X_train, X_test, y_train, y_test
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train = np.resize(X_train,(2499,300,300,3))
X_test = np.resize(X_test,(625,300,300,3))


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

(2499, 270000)
(625, 270000)
(2499,)
(625,)


In [ ]:
# for image_batch, labels_batch in train_dataset:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break

# AUTOTUNE = tf.data.AUTOTUNE

# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# normalization_layer = layers.Rescaling(1./255)

# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]
# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))



In [9]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [12]:
num_classes = 11


model = Sequential([
  layers.Rescaling(1./255, input_shape=(300,300,3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_dataset,
  validation_data=test_dataset,
  epochs=epochs
)

Epoch 1/10
40/40 [==============================] - 105s 3s/step - loss: 2.2971 - accuracy: 0.1373
Epoch 2/10
40/40 [==============================] - 125s 3s/step - loss: 2.0648 - accuracy: 0.2745
Epoch 3/10
40/40 [==============================] - 123s 3s/step - loss: 1.8660 - accuracy: 0.3689
Epoch 4/10
40/40 [==============================] - 121s 3s/step - loss: 1.7199 - accuracy: 0.4134
Epoch 5/10
 5/40 [==>...........................] - ETA: 2:04 - loss: 1.5729 - accuracy: 0.4406

In [96]:

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [103]:
num_classes = 10


model = Sequential([
  layers.Rescaling(1./255, input_shape=(300,300,3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [104]:
train_dataset


<PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.float64)>

In [89]:
epochs=10
history = model.fit(
  train_dataset,
  epochs=epochs
)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\doron\anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\doron\anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\doron\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\doron\anaconda3\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\doron\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\doron\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer

In [19]:
X_train

array([[-0.40325955, -0.70679855, -0.7421356 , ..., -0.32972956,
        -0.28070062, -0.01989093],
       [ 0.03374127, -0.32078636, -0.6471267 , ..., -0.5550537 ,
        -0.49008387, -0.14747787],
       [-0.405304  , -0.7505136 , -0.8592299 , ..., -0.55013895,
        -0.47980505, -0.36489233],
       ...,
       [-0.13288125, -0.16036034, -0.23167104, ..., -0.5043936 ,
        -0.30163893, -0.2848027 ],
       [-0.13288125, -0.5966861 , -0.8338108 , ..., -0.3803897 ,
        -0.43640563, -0.38675302],
       [-0.3861373 ,  0.07883527,  0.7588385 , ...,  3.2690299 ,
        -0.40747267, -0.3931125 ]], dtype=float32)

In [18]:
epochs=10
history = model.fit(
  X_train,
  validation_data=y_train,
  epochs=epochs
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()